# American put option pricing using Binomial model

Import required modules and define the $^{n}C_{p} \equiv $ choosing $p$ objects from $n$ objects, where the order of selection doesn't matter 

In [1]:
import numpy as np
def nCp(m,r1):
    return np.math.factorial(m)/(np.math.factorial(r1)*np.math.factorial(m-r1))

## Black Scholes Parameters
* Annualized volatility of the stock $\sigma$,
* Risk free interest rate $r \%$,
## Other parametes
* Initial stock price $S_0$, 
* Time to expiry $T$ years, 
* Total nodes of the Binomial lattice $n_{node}$,
* Nodes we want to evaluate at $n$,
* Strike price $K$,
* Dividend yield $c \%$ 
## Binomial model parameters
* $R_n = \exp(r\frac{T}{n})$ 
* $R_n - c_n = \exp((r-c)\frac{T}{n})$ 
* $u_n = \exp(\sigma\sqrt{\frac{T}{n}})$
* $d_n = 1/u_n$
* Risk neutral probablility $q_n = \frac{R_n-c_n-d_n}{u_n-d_n}$

In [2]:
#
T = 0.25
S0 = 100
r = 0.02
sigma = 0.3
c = 0.01
n = 15
K = 110
n_node = 15
#
R = np.exp(r*T/n)
R_c = np.exp((r-c)*T/n)
u = np.exp(sigma*np.sqrt(T/n))
d = np.exp(-sigma*np.sqrt(T/n))
q = (np.exp((r-c)*T/n) - d)/(u -d)

## Difference between the stock and strike price at each nodes

In [ ]:
# Stock price
def Sn(S0,n):
    return  [round(S0 * u**i * d**(n-i),3) for i in reversed(range(n+1))]
# Stock price tree 
def Sn_tree(S0,n):
    return  [Sn(S0,n) for n in range(n+1)]
# Differnece between the strike price K and stock price S0 at node n
def KSndiff(S0,K,n):
    return [round(max(0,K-ele),3) for ele in Sn(S0, n)]
# Strike price - stock price tree
def KSndiff_tree(S0,K,n):
    return [KSndiff(S0,K,n) for n in range(n+1)]
# Pairwise sum
def pairwise_q_sum(lst):
    sum = 0;
    pair_sum = []
    for i in range(len(lst)-1):
        sum = (1/R)*(q*lst[i] + (1-q)*lst[i+1])
        pair_sum.append(round(sum,3))
    return pair_sum

In [3]:
print(KSndiff(S0,K,n_node-1))
print(pairwise_q_sum(KSndiff(S0,K,n_node)))

[0, 0, 0, 0, 0, 0, 1.946, 10.0, 17.454, 24.352, 30.735, 36.643, 42.111, 47.171, 51.854]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.07, 9.98, 17.432, 24.329, 30.712, 36.619, 42.086, 47.145, 51.827]


## Price of American put at the starting node

* American put option price for the above parameters is 12.30

In [4]:
put_A = [None]*(n_node+1)
put_A[n_node] = KSndiff(S0,K,n_node)
put_A[n_node-1] = max(KSndiff(S0,K,n_node-1),pairwise_q_sum(KSndiff(S0,K,n_node)))
#put_A[n_node-2] = max(KSndiff(S0,K,n_node-2),put_A[n_node-1])
for n in range(2,n_node+1):
    put_A[n_node-n] = max(KSndiff(S0,K,n_node-n),pairwise_q_sum(put_A[n_node+1-n]))

In [5]:
put_A

[[12.306],
 [9.533, 15.005],
 [7.008, 11.989, 17.942],
 [4.813, 9.142, 14.759, 21.043],
 [3.02, 6.556, 11.657, 17.779, 24.223],
 [1.672, 4.329, 8.722, 14.512, 20.96, 27.405],
 [0.772, 2.546, 6.061, 11.309, 17.629, 24.206, 30.528],
 [0.267, 1.263, 3.793, 8.266, 14.269, 20.9, 27.43, 33.555],
 [0.052, 0.475, 2.028, 5.508, 10.947, 17.502, 24.211, 30.571, 36.473],
 [0.0, 0.103, 0.836, 3.186, 7.764, 14.042, 20.871, 27.467, 33.603, 39.282],
 [0.0,
  0.0,
  0.203,
  1.451,
  4.872,
  10.575,
  17.415,
  24.239,
  30.618,
  36.522,
  41.986],
 [0.0,
  0.0,
  0.0,
  0.401,
  2.471,
  7.204,
  13.853,
  20.882,
  27.513,
  33.651,
  39.332,
  44.588],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.79,
  4.103,
  10.217,
  17.39,
  24.284,
  30.665,
  36.571,
  42.036,
  47.093],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.558,
  6.575,
  13.758,
  20.925,
  27.559,
  33.699,
  39.381,
  44.639,
  49.505],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  3.07,
  9.98,
  17.432,
  24.329,
  30.712,
  36.619,
  42.086,
  47